# Spark ML для задачи оттока

Источник данных: [Kaggle](https://www.kaggle.com/sakshigoyal7/credit-card-customers)

Описания полей:

Название | Описание
:--------|:--------
CLIENTNUM | Client number. Unique identifier for the customer holding the account
Attrition_Flag | Internal event (customer activity) variable
Customer_Age | Demographic variable - Customer's Age in Years
Gender | Demographic variable - M=Male, F=Female
Dependent_count | Demographic variable - Number of dependents
Education_Level | Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)
Marital_Status | Demographic variable - Married, Single, Divorced, Unknown
Income_Category | Demographic variable - Annual Income Category of the account holder (< 40K, 40K - 60K, 60K - 80K, 80K-120K, > 120K, Unknown)
Card_Category | Product Variable - Type of Card (Blue, Silver, Gold, Platinum)
Months_on_book | Period of relationship with bank

In [1]:
import $ivy.`org.apache.spark:spark-sql_2.12:3.0.1`
import $ivy.`org.apache.spark:spark-mllib_2.12:3.0.1`
import $ivy.`sh.almond:almond-spark_2.12:0.10.9`

import $ivy.$                                      

import $ivy.$                                        

import $ivy.$                                   

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
      SparkSession.builder()
        .master("local[*]")
        .getOrCreate()
    }

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@6811e244
import spark.implicits._

## Загрузка и знакомство с данными

In [4]:
val raw = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv("BankChurners.csv")

val columns: Array[String] = raw.columns
val columnsLen: Int = columns.length
val colsToDrop: Array[String] = columns.slice(columnsLen - 2, columnsLen) ++ Array("CLIENTNUM")

val df = raw.drop(colsToDrop: _*)

df.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

raw: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]
columns: Array[String] = Array(
  "CLIENTNUM",
  "Attrition_Flag",
  "Customer_Age",
  "Gender",
  "Dependent_count",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
)
columnsLen: Int = 23
colsToDrop: Array[String] = Array(
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Leve

### Определяем типы колонок

In [5]:
df.printSchema

root
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: double (nullable = true)



In [6]:
df.dtypes.groupBy(_._2).mapValues(_.length)

res5: Map[String, Int] = Map(
  "DoubleType" -> 5,
  "StringType" -> 6,
  "IntegerType" -> 9
)

### Проверяем числовые колонки

In [7]:
val numericColumns = df.dtypes.filter(!_._2.equals("StringType")).map(d => d._1)
df.select(numericColumns.map(col): _*).summary().show

+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|summary|     Customer_Age|   Dependent_count|    Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|     Credit_Limit|Total_Revolving_Bal|  Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|  Total_Trans_Amt|   Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|  count|            10127|             10127|             10127|                   10127|                 10127|              

numericColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

In [8]:
df.groupBy($"Customer_Age").count().show(100)

+------------+-----+
|Customer_Age|count|
+------------+-----+
|          31|   91|
|          65|  101|
|          53|  387|
|          34|  146|
|          28|   29|
|          26|   78|
|          27|   32|
|          44|  500|
|          47|  479|
|          52|  376|
|          40|  361|
|          57|  223|
|          54|  307|
|          48|  472|
|          64|   43|
|          41|  379|
|          43|  473|
|          37|  260|
|          61|   93|
|          35|  184|
|          59|  157|
|          55|  279|
|          39|  333|
|          49|  495|
|          51|  398|
|          63|   65|
|          50|  452|
|          45|  486|
|          38|  303|
|          73|    1|
|          70|    1|
|          62|   93|
|          29|   56|
|          32|  106|
|          60|  127|
|          56|  262|
|          58|  157|
|          33|  127|
|          68|    2|
|          42|  426|
|          30|   70|
|          66|    2|
|          46|  490|
|          67|    4|
|          36

## Целевая колонка

In [9]:
val dft = df.withColumn("target", when($"Attrition_Flag" === "Existing Customer", 0).otherwise(1))

dft.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+---------

dft: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

### Несбалансированное распределение данных

In [10]:
dft.groupBy("target").count.show

+------+-----+
|target|count|
+------+-----+
|     1| 1627|
|     0| 8500|
+------+-----+



### Oversampling

In [11]:
val df1 = dft.filter($"target" === 1)
val df0 = dft.filter($"target" === 0)

val df1count = df1.count
val df0count = df0.count

df0count / df1count

df1: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df0: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df1count: Long = 1627L
df0count: Long = 8500L
res10_4: Long = 5L

In [12]:
val df1Over = df1
        .withColumn("dummy", explode(lit((1 to (df0count / df1count).toInt).toArray)))
        .drop("dummy")

df1Over.show(10, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+---------

df1Over: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

In [13]:
val data = df0.unionAll(df1Over)
data.groupBy("target").count.show

+------+-----+
|target|count|
+------+-----+
|     1| 8135|
|     0| 8500|
+------+-----+



data: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

## Работа с признаками

### Проверяем корреляции числовых признаков

In [14]:
val pairs = numericColumns
        .flatMap(f1 => numericColumns.map(f2 => (f1, f2)))
        .filter { p => !p._1.equals(p._2) }
        .map { p => if (p._1 < p._2) (p._1, p._2) else (p._2, p._1) }
        .distinct

val corr = pairs
        .map { p => (p._1, p._2, data.stat.corr(p._1, p._2)) }
        .filter(_._3 > 0.6)

pairs: Array[(String, String)] = Array(
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Customer_Age"),
  ("Credit_Limit", "Customer_Age"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Customer_Age"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Avg_Utilization_Ratio", "Customer_Age"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Dependent_count"),
  ("Credit_Limit", "Dependent_count"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Dependent_count"),
  ("Dependent_count", "Total_Amt_Chng_Q4_Q1"),
  ("Dependent_count", "Total_Trans_Amt"),
  ("Dependen

In [15]:
corr.sortBy(_._3).reverse.foreach { c => println(s"${c._1}\t\t${c._2}\t\t${c._3}") }

Avg_Open_To_Buy		Credit_Limit		0.9952040726156253
Total_Trans_Amt		Total_Trans_Ct		0.8053901681243808
Customer_Age		Months_on_book		0.7805047706891142
Avg_Utilization_Ratio		Total_Revolving_Bal		0.6946855441968229


In [16]:
val numericColumnsFinal = numericColumns.diff(corr.map(_._2))

numericColumnsFinal: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

### Категориальные признаки

#### Индексируем строковые колонки

In [17]:
import org.apache.spark.ml.feature.StringIndexer

val stringColumns = data
        .dtypes
        .filter(_._2.equals("StringType"))
        .map(_._1)
        .filter(!_.equals("Attrition_Flag"))

val stringColumnsIndexed = stringColumns.map(_ + "_Indexed")

val indexer = new StringIndexer()
        .setInputCols(stringColumns)
        .setOutputCols(stringColumnsIndexed)

val indexed = indexer.fit(data).transform(data)
indexed.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Marital_Status_Indexed|Income_Category_Indexed|Gender_Indexed|Education_Level_Indexed|Card_Category_Indexed|
+-----------------+------------+------+---------------+---------------+---------

import org.apache.spark.ml.feature.StringIndexer


stringColumns: Array[String] = Array(
  "Gender",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category"
)
stringColumnsIndexed: Array[String] = Array(
  "Gender_Indexed",
  "Education_Level_Indexed",
  "Marital_Status_Indexed",
  "Income_Category_Indexed",
  "Card_Category_Indexed"
)
indexer: StringIndexer = strIdx_c2e309055038
indexed: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 24 more fields]

#### Кодируем категориальные признаки

In [18]:
import org.apache.spark.ml.feature.OneHotEncoder

val catColumns = stringColumnsIndexed.map(_ + "_Coded")
    
val encoder = new OneHotEncoder()
        .setInputCols(stringColumnsIndexed)
        .setOutputCols(catColumns)

val encoded = encoder.fit(indexed).transform(indexed)
encoded.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Marital_Status_Indexed|Income_Category_Indexed|Gend

import org.apache.spark.ml.feature.OneHotEncoder


catColumns: Array[String] = Array(
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
encoder: OneHotEncoder = oneHotEncoder_3083310f4ee1
encoded: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 29 more fields]

### Собираем признаки в вектор

In [19]:
import org.apache.spark.ml.feature.VectorAssembler

val featureColumns = numericColumnsFinal ++ catColumns

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembled = assembler.transform(encoded)
assembled.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------------------------------------------------------------------------------------------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Am

import org.apache.spark.ml.feature.VectorAssembler


featureColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
assembler: VectorAssembler = VectorAssembler: uid=vecAssembler_323bd81ae46a, handleInvalid=error, numInputCols=15
assembled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 30 more fields]

### Нормализация

In [20]:
import org.apache.spark.ml.feature.MinMaxScaler

val scaler = new MinMaxScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")

val scaled = scaler.fit(assembled).transform(assembled)
scaled.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|E

import org.apache.spark.ml.feature.MinMaxScaler


scaler: MinMaxScaler = minMaxScal_0583594a8218
scaled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 31 more fields]

## Feature Selection (отбор признаков)

In [21]:
import org.apache.spark.ml.feature.ChiSqSelector

val selector = new ChiSqSelector()
  .setNumTopFeatures(10)
  .setFeaturesCol("scaledFeatures")
  .setLabelCol("target")
  .setOutputCol("selectedFeatures")

val dataF = selector.fit(scaled).transform(scaled)
dataF.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------

import org.apache.spark.ml.feature.ChiSqSelector


selector: ChiSqSelector = chiSqSelector_cd73554525dd
dataF: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

## Моделирование

### Обучающая и тестовая выборки

In [22]:
val tt = dataF.randomSplit(Array(0.7, 0.3))
val training = tt(0)
val test = tt(1)

println(s"training\t${training.count}\ntest\t${test.count}")

training	11732
test	4903


tt: Array[Dataset[Row]] = Array(
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields],
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
)
training: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
test: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

### Логистическая регрессия

In [23]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression()
        .setMaxIter(1000)
        .setRegParam(0.2)
        .setElasticNetParam(0.8)
        .setFeaturesCol("selectedFeatures")
        .setLabelCol("target")

val lrModel = lr.fit(training)

println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

21/01/30 13:14:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/01/30 13:14:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/01/30 13:14:40 INFO OWLQN: Step Size: 0,1463
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692577 (rel: 0,000210) 0,0242979
21/01/30 13:14:40 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692508 (rel: 9,95e-05) 0,0304871
21/01/30 13:14:40 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692448 (rel: 8,64e-05) 0,0153372
21/01/30 13:14:40 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692418 (rel: 4,40e-05) 0,0176589
21/01/30 13:14:40 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692396 (rel: 3,13e-05) 0,0104927
21/01/30 13:14:40 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:40 INFO OWLQN: Val and Grad Norm: 0,692377 (rel: 2,74e-05) 0,0118180
21

Coefficients: (10,[7],[-1.7920856148299178]) Intercept: 0.25501100216440425


import org.apache.spark.ml.classification.LogisticRegression


lr: LogisticRegression = logreg_eaf3838d71b1
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_eaf3838d71b1, numClasses=2, numFeatures=10

### Training Summary

In [24]:
val trainingSummary = lrModel.binarySummary
println(s"accuracy: ${trainingSummary.accuracy}")
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")

accuracy: 0.7006478008864644
areaUnderROC: 0.7480135302596236


trainingSummary: org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@88e8e07

### Проверяем модель на тестовой выборке

In [25]:
val predicted = lrModel.transform(test)
predicted.select("target", "rawPrediction", "probability", "prediction").show(10, truncate = false)

+------+------------------------------------------+----------------------------------------+----------+
|target|rawPrediction                             |probability                             |prediction|
+------+------------------------------------------+----------------------------------------+----------+
|0     |[0.05139028093117942,-0.05139028093117942]|[0.5128447434850257,0.48715525651497427]|0.0       |
|0     |[-0.08371579665427481,0.08371579665427481]|[0.47908326536604856,0.5209167346339514]|1.0       |
|0     |[0.07117367086333526,-0.07117367086333526]|[0.5177859101885983,0.48221408981140157]|0.0       |
|0     |[-0.10446422999775526,0.10446422999775526]|[0.47390766650047045,0.5260923334995296]|1.0       |
|0     |[-0.03208597414840475,0.03208597414840475]|[0.4919791945758656,0.5080208054241345] |1.0       |
|0     |[-0.0571771028428463,0.0571771028428463]  |[0.48570961727889,0.5142903827211099]   |1.0       |
|0     |[0.13100636236546503,-0.13100636236546503]|[0.5327048287

predicted: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 35 more fields]

In [26]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator().setLabelCol("target")

println(s"areaUnderROC: ${evaluator.evaluate(predicted)}\n")

areaUnderROC: 0.7383373102241036



import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


evaluator: BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_8f06c4d8be3e, metricName=areaUnderROC, numBins=1000

#### Confusion Matrix (матрица ошибок)

* True Positive (TP) - label is positive and prediction is also positive
* True Negative (TN) - label is negative and prediction is also negative
* False Positive (FP) - label is negative but prediction is positive
* False Negative (FN) - label is positive but prediction is negative

In [27]:
val tp = predicted.filter(($"target" === 1) and ($"prediction" === 1)).count
val tn = predicted.filter(($"target" === 0) and ($"prediction" === 0)).count
val fp = predicted.filter(($"target" === 0) and ($"prediction" === 1)).count
val fn = predicted.filter(($"target" === 1) and ($"prediction" === 0)).count

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t$tn\n")

Confusion Matrix:
1166	277
1274	2186



tp: Long = 1166L
tn: Long = 2186L
fp: Long = 277L
fn: Long = 1274L

#### Accuracy, Precision, Recall

* Accuracy (доля правильных ответов) = TP + TN / TP + TN + FP + FN
* Precision (точность) = TP / TP + FP
* Recall (полнота) = TP / TP + FN

In [28]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.6836630634305527
Precision = 0.808038808038808
Recall = 0.4778688524590164



accuracy: Double = 0.6836630634305527
precision: Double = 0.808038808038808
recall: Double = 0.4778688524590164

### Настраиваем модель (подбираем гиперпараметры)

In [29]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.01, 0.1, 0.5))
  .addGrid(lr.fitIntercept)
  .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
  .build()

val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(lr)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setTrainRatio(0.7)
  .setParallelism(2)

val model = trainValidationSplit.fit(dataF)

21/01/30 13:14:46 INFO StrongWolfeLineSearch: Line search t: 1.1178981010068636 fval: 0.6319393949326716 rhs: 0.6927715463534929 cdd: 0.0011184221443388625
21/01/30 13:14:46 INFO LBFGS: Step Size: 1,118
21/01/30 13:14:46 INFO LBFGS: Val and Grad Norm: 0,631939 (rel: 0,0878) 0,888493
21/01/30 13:14:46 INFO OWLQN: Step Size: 0,1468
21/01/30 13:14:46 INFO OWLQN: Val and Grad Norm: 0,678266 (rel: 0,0210) 0,304465
21/01/30 13:14:46 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:46 INFO OWLQN: Val and Grad Norm: 0,570451 (rel: 0,159) 0,356071
21/01/30 13:14:46 INFO LBFGS: Step Size: 1,000
21/01/30 13:14:46 INFO LBFGS: Val and Grad Norm: 0,550542 (rel: 0,129) 0,376918
21/01/30 13:14:46 INFO LBFGS: Step Size: 1,000
21/01/30 13:14:46 INFO LBFGS: Val and Grad Norm: 0,521130 (rel: 0,0534) 0,0671982
21/01/30 13:14:46 INFO LBFGS: Step Size: 1,000
21/01/30 13:14:46 INFO LBFGS: Val and Grad Norm: 0,517558 (rel: 0,00686) 0,0505441
21/01/30 13:14:46 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:46 INFO OWL

21/01/30 13:14:48 INFO OWLQN: Step Size: 0,1468
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,679448 (rel: 0,0193) 0,291989
21/01/30 13:14:48 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,583195 (rel: 0,142) 0,370425
21/01/30 13:14:48 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,522132 (rel: 0,000174) 0,0234388
21/01/30 13:14:48 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,521823 (rel: 0,000590) 0,0330782
21/01/30 13:14:48 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,570759 (rel: 0,0213) 0,241592
21/01/30 13:14:48 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,521666 (rel: 0,000301) 0,0709613
21/01/30 13:14:48 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:48 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:48 INFO OWLQN: Val and Grad Norm: 0,564566 (rel: 0,0108) 0,170597
21/01/30 13:14:48 INFO OWLQN: Val and Grad 

21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,517473 (rel: 1,79e-05) 0,0244084
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,1250
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,531625 (rel: 0,000265) 0,0419761
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,517419 (rel: 0,000103) 0,0299235
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,531595 (rel: 5,63e-05) 0,0311413
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,517374 (rel: 8,61e-05) 0,0145936
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,531536 (rel: 0,000111) 0,0827145
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,2500
21/01/30 13:14:50 INFO OWLQN: Val and Grad Norm: 0,517342 (rel: 6,21e-05) 0,0245317
21/01/30 13:14:50 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:50 INFO OWLQN

21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 4,75e-08) 0,000632037
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,529262 (rel: 9,59e-05) 0,0239066
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 8,68e-08) 0,000427459
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,529243 (rel: 3,61e-05) 0,0201443
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 3,19e-08) 0,000392308
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,529203 (rel: 7,48e-05) 0,0205280
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 2,86e-08) 0,000331144
21/01/30 13:14:52 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:52 IN

21/01/30 13:14:53 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:53 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 1,25e-09) 6,03246e-05
21/01/30 13:14:53 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:53 INFO OWLQN: Val and Grad Norm: 0,528856 (rel: 6,02e-06) 0,00996669
21/01/30 13:14:54 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:54 INFO OWLQN: Val and Grad Norm: 0,528852 (rel: 8,41e-06) 0,0102706
21/01/30 13:14:54 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:54 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 1,34e-09) 8,25658e-05
21/01/30 13:14:54 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:54 INFO OWLQN: Val and Grad Norm: 0,528840 (rel: 2,27e-05) 0,00425547
21/01/30 13:14:54 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:54 INFO OWLQN: Val and Grad Norm: 0,528834 (rel: 1,11e-05) 0,00308100
21/01/30 13:14:54 INFO OWLQN: Step Size: 0,5000
21/01/30 13:14:54 INFO OWLQN: Val and Grad Norm: 0,516563 (rel: 5,29e-10) 5,52650e-05
21/01/30 13:14:54 INFO OWLQN: Step Size: 1,000
21/01/30 13:14:54 INFO O

...

21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,562803 (rel: 0,00318) 0,0318279
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,561014 (rel: 0,00318) 0,00687583
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,560923 (rel: 0,000161) 0,000814142
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,560923 (rel: 9,87e-07) 0,000456640
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,560923 (rel: 3,84e-07) 7,20345e-05
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,560923 (rel: 1,55e-08) 3,01901e-05
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFGS: Val and Grad Norm: 0,560923 (rel: 3,91e-09) 3,56014e-06
21/01/30 13:15:12 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:12 INFO LBFG

21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,676542 (rel: 1,71e-07) 0,000611829
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,636078 (rel: 3,83e-08) 0,000226578
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,676542 (rel: 1,93e-07) 0,000749336
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,636078 (rel: 1,94e-08) 0,000171183
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,676542 (rel: 2,60e-07) 0,000543948
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,636078 (rel: 4,18e-08) 0,000108471
21/01/30 13:15:13 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:13 INFO OWLQN: Val and Grad Norm: 0,636078 (rel: 2,84e-08) 0,000167274
21/01/30 13:15:13 INFO OWLQN: Step Size: 0,5000
21/01/30 13:15:13 INF

21/01/30 13:15:14 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:14 INFO OWLQN: Val and Grad Norm: 0,676541 (rel: 3,96e-13) 1,08811e-06
21/01/30 13:15:14 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:14 INFO OWLQN: Val and Grad Norm: 0,676541 (rel: 8,50e-14) 7,84402e-07
21/01/30 13:15:14 INFO OWLQN: Step Size: 1,000
21/01/30 13:15:14 INFO OWLQN: Val and Grad Norm: 0,676541 (rel: 6,60e-14) 4,70327e-07
21/01/30 13:15:14 INFO OWLQN: Converged because gradient converged
21/01/30 13:15:15 INFO OWLQN: Converged because gradient converged
21/01/30 13:15:15 INFO StrongWolfeLineSearch: Line search t: 0.6926436411633152 fval: 0.6533712794724852 rhs: 0.6927764771545549 cdd: 4.0594802958929407E-4
21/01/30 13:15:15 INFO LBFGS: Step Size: 0,6926
21/01/30 13:15:15 INFO LBFGS: Val and Grad Norm: 0,653371 (rel: 0,0569) 0,578588
21/01/30 13:15:15 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:15 INFO LBFGS: Val and Grad Norm: 0,628930 (rel: 0,0374) 0,101760
21/01/30 13:15:15 INFO LBFGS: Step Size: 1,000
21/01/3

21/01/30 13:15:17 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:17 INFO LBFGS: Val and Grad Norm: 0,504931 (rel: 2,73e-10) 3,02990e-06
21/01/30 13:15:17 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:17 INFO LBFGS: Val and Grad Norm: 0,504931 (rel: 7,87e-11) 7,08950e-07
21/01/30 13:15:17 INFO StrongWolfeLineSearch: Line search t: 0.3593667548004782 fval: 0.5049306246222138 rhs: 0.5049306246228531 cdd: -1.1237904468019169E-16
21/01/30 13:15:17 INFO LBFGS: Step Size: 0,3594
21/01/30 13:15:17 INFO LBFGS: Val and Grad Norm: 0,504931 (rel: 1,27e-12) 2,04092e-06
21/01/30 13:15:17 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:17 INFO LBFGS: Val and Grad Norm: 0,504931 (rel: 1,19e-12) 8,86215e-08
21/01/30 13:15:17 INFO LBFGS: Converged because gradient converged


import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}


paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	logreg_eaf3838d71b1-elasticNetParam: 0.0,
	logreg_eaf3838d71b1-fitIntercept: true,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 0.5,
	logreg_eaf3838d71b1-fitIntercept: true,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 1.0,
	logreg_eaf3838d71b1-fitIntercept: true,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 0.0,
	logreg_eaf3838d71b1-fitIntercept: false,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 0.5,
	logreg_eaf3838d71b1-fitIntercept: false,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 1.0,
	logreg_eaf3838d71b1-fitIntercept: false,
	logreg_eaf3838d71b1-regParam: 0.01
},
  {
	logreg_eaf3838d71b1-elasticNetParam: 0.0,
	logreg_eaf3838d71b1-fitIntercept: true,
	logreg_

In [30]:
model.bestModel.extractParamMap()

res29: org.apache.spark.ml.param.ParamMap = {
	logreg_eaf3838d71b1-aggregationDepth: 2,
	logreg_eaf3838d71b1-elasticNetParam: 0.0,
	logreg_eaf3838d71b1-family: auto,
	logreg_eaf3838d71b1-featuresCol: selectedFeatures,
	logreg_eaf3838d71b1-fitIntercept: true,
	logreg_eaf3838d71b1-labelCol: target,
	logreg_eaf3838d71b1-maxIter: 1000,
	logreg_eaf3838d71b1-predictionCol: prediction,
	logreg_eaf3838d71b1-probabilityCol: probability,
	logreg_eaf3838d71b1-rawPredictionCol: rawPrediction,
	logreg_eaf3838d71b1-regParam: 0.01,
	logreg_eaf3838d71b1-standardization: true,
	logreg_eaf3838d71b1-threshold: 0.5,
	logreg_eaf3838d71b1-tol: 1.0E-6
}

In [31]:
val bestML = new LogisticRegression()
        .setMaxIter(1000)
        .setRegParam(0.01)
        .setElasticNetParam(0.0)
        .setFeaturesCol("selectedFeatures")
        .setLabelCol("target")

bestML: LogisticRegression = logreg_0648cd3d9e7f

## Собираем всё вместе (Pipeline)

1. Отобрали числовые признаки: numericColumnsFinal
2. Проиндексировали строковые признаки: indexer
3. Закодировали категориальные признки: encoder
4. Собрали признаки в вектор: assembler
5. Нормализовали признаки: scaler
6. Провели отбор признаков: selector
7. Рассчитали модель: bestML

In [32]:
import org.apache.spark.ml.Pipeline

val pipeline = new Pipeline().setStages(Array(indexer, encoder, assembler, scaler, selector, bestML))

import org.apache.spark.ml.Pipeline


pipeline: Pipeline = pipeline_8e25609a8360

In [33]:
val ttData = data.randomSplit(Array(0.7, 0.3))
val trainingData = ttData(0)
val testData = ttData(1)

val pipelineModel = pipeline.fit(trainingData)

21/01/30 13:15:19 INFO StrongWolfeLineSearch: Line search t: 1.0839137451838616 fval: 0.6351667053803528 rhs: 0.6928285232187219 cdd: 0.0011142331008877008
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,084
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,635167 (rel: 0,0832) 0,880028
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,556800 (rel: 0,123) 0,397917
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,524839 (rel: 0,0574) 0,0680475
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,521263 (rel: 0,00681) 0,0503016
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,517784 (rel: 0,00667) 0,0409918
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO LBFGS: Val and Grad Norm: 0,512974 (rel: 0,00929) 0,00959991
21/01/30 13:15:19 INFO LBFGS: Step Size: 1,000
21/01/30 13:15:19 INFO

ttData: Array[Dataset[Row]] = Array(
  [Attrition_Flag: string, Customer_Age: int ... 19 more fields],
  [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
)
trainingData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
testData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_8e25609a8360

## Сохраняем модель

In [34]:
pipelineModel.write.overwrite.save("pipelineModel")